In [2]:
# coding: utf-8

import pandas as pd
from bs4 import BeautifulSoup
from config import GROUP_DICT, MAX_PAGE, SQL_DICT, HEADERS, PROXY_POOL_URL, MAX_GET_RETRY, OUTPUT_PATH
from base import _Sql_Base
import requests
import emoji
import time
import random
import logging
import os

class HTTPError(Exception):

    """ HTTP状态码不是200异常 """

    def __init__(self, status_code, url):
        self.status_code = status_code
        self.url = url

    def __str__(self):
        return "%s HTTP %s" % (self.url, self.status_code)
    

def get_logger(name):
    """logger
    """
    default_logger = logging.getLogger(name)
    default_logger.setLevel(logging.DEBUG)
    stream = logging.StreamHandler()
    stream.setLevel(logging.DEBUG)
    formatter = logging.Formatter("[%(levelname)s] %(asctime)s - %(message)s")
    stream.setFormatter(formatter)
    default_logger.addHandler(stream)
    return default_logger

logger = get_logger("douban_spider")

class Douban_corpus_spider(_Sql_Base):

    def __init__(self, is_proxy = False):

        self.GROUP_DICT = GROUP_DICT
        self.MAX_PAGE = MAX_PAGE
        self.sql_engine = self.create_engine(SQL_DICT)
        self.is_proxy = is_proxy
        if is_proxy:
            self.proxyIP = self.get_proxy()
            
    def request_douban(self, url):
       # 从代理池中随机取出一个IP
        headers = {
            'User-Agent': HEADERS
        }
        for i in range(MAX_GET_RETRY):
            try:
                if self.is_proxy:
                    proxyIP = self.proxyIP
                    proxies = {
                        'http' : proxyIP,
                        'https': proxyIP
                    }
                    response = requests.get(url, proxies=proxies, headers=headers)
                else:
                    response = requests.get(url, headers=headers)
                if response.status_code != 200:
                    raise HTTPError(response.status_code, url)
                else:
                    print('proxy: %s成功获得数据 %s' %(self.proxyIP, url))
                break
            except Exception as exc:
                logger.warn("%s %d failed!\n%s", url, i, str(exc))
                if self.is_proxy:
                    self.proxyIP = self.get_proxy()
                continue
        return response.text

    def get_proxy(self):
        try:
            response = requests.get(PROXY_POOL_URL)
            if response.status_code == 200:
                print('proxy: %s' %response.text)
                return "http://%s" %response.text
        except ConnectionError:
            return None

    def spider_links(self, group, page):
        url = '{}discussion?start={}'.format(self.GROUP_DICT[group], str(page*25))
        html = self.request_douban(url)
        soup = BeautifulSoup(html, 'lxml')
        list_ = soup.find(class_='olt').find_all('tr')
        page_link = []; page_title = []
        for item in list_:
            try:
                page_link.append(item.find('a').get('href'))
                page_title.append(item.find('a').get('title'))
            except:
                continue
        return page_link, page_title

    def spider_page(self, url):
        html = self.request_douban(url)
        soup = BeautifulSoup(html, 'lxml')
        for item in soup.find(type="application/ld+json"):
            try:
                page_author_diag = json.loads(item)['text']
            except:
                page_author_diag = ''
        list_ = soup.find_all(class_='clearfix comment-item reply-item')
        page_comments = []
        for item in list_:
            try:
                page_comments.append(item.find('p').contents[0])
            except:
                continue
        return page_author_diag, page_comments

    def spider_group(self, group):
        spider_outputs = {}
        link_list = []
        title_list = []
        for page in range(self.MAX_PAGE):
            link_list_page, title_list_page = self.spider_links(group, page)
            link_list = link_list + link_list_page
            title_list = title_list + title_list_page
        for link in link_list:
            spider_outputs[link] = {}
            spider_outputs[link]['title'] = title_list[link_list.index(link)]
            spider_outputs[link]['author_diag'], spider_outputs[link]['comments'] = self.spider_page(link)
            self.json_write(spider_outputs, os.path.join(OUTPUT_PATH, '{}.json'.format(group)))
        return spider_outputs

    def group_dict_transfer(self, output_dict):
        data = pd.DataFrame(output_dict).T
        data['link'] = data.index
        data = data.reset_index(drop = True)[['link','title','author_diag','comments']]
        def comments_sub(a):
            b = ''
            for item in a:
                b = item + '|' + b
            return b
        data['comments'] = data['comments'].apply(comments_sub)
        for col in ['title','author_diag','comments']:
            data[col] = data[col].apply(emoji.demojize)
        return data

    def run(self):
        for group in self.GROUP_DICT.keys():
            output_dict = self.spider_group(group)
            output_table = self.group_dict_transfer(output_dict)
            self.table_save(output_table, group)

In [3]:
dcs = Douban_corpus_spider(is_proxy = True)

output_dict = dcs.spider_group('互相表扬夸夸小组')

proxy: 120.27.208.131:80
proxy: http://120.27.208.131:80成功获得数据
proxy: http://120.27.208.131:80成功获得数据


C:\Users\kenshinpg\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
[WARNING] 2019-05-29 17:13:39,485 - https://www.douban.com/group/topic/138418608/ 0 failed!
HTTPSConnectionPool(host='www.douban.com', port=443): Max retries exceeded with url: /group/topic/138418608/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000020B20092B70>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',)))
[WARNING] 2019-05-29 17:13:39,485 - https://www.douban.com/group/topic/138418608/ 0 failed!
HTTPSConnectionPool(host='www.douban.com', port=443): Max retries exceeded with url: /group/topic/138418608/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000020B20092B70>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时

proxy: 109.195.227.244:8080


[WARNING] 2019-05-29 17:14:01,499 - https://www.douban.com/group/topic/138418608/ 1 failed!
HTTPSConnectionPool(host='www.douban.com', port=443): Max retries exceeded with url: /group/topic/138418608/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000020B20092DD8>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',)))
[WARNING] 2019-05-29 17:14:01,499 - https://www.douban.com/group/topic/138418608/ 1 failed!
HTTPSConnectionPool(host='www.douban.com', port=443): Max retries exceeded with url: /group/topic/138418608/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000020B20092DD8>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',)))


proxy: 203.150.160.132:8080


...

proxy: 47.62.123.175:8197


[WARNING] 2019-05-29 17:14:27,350 - https://www.douban.com/group/topic/138418608/ 4 failed!
('Connection aborted.', BadStatusLine('<html>\r\n',))
[WARNING] 2019-05-29 17:14:27,350 - https://www.douban.com/group/topic/138418608/ 4 failed!
('Connection aborted.', BadStatusLine('<html>\r\n',))


proxy: 203.150.160.132:8080


[WARNING] 2019-05-29 17:14:49,368 - https://www.douban.com/group/topic/138418608/ 5 failed!
HTTPSConnectionPool(host='www.douban.com', port=443): Max retries exceeded with url: /group/topic/138418608/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000020B200924E0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',)))
[WARNING] 2019-05-29 17:14:49,368 - https://www.douban.com/group/topic/138418608/ 5 failed!
HTTPSConnectionPool(host='www.douban.com', port=443): Max retries exceeded with url: /group/topic/138418608/ (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000020B200924E0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',)))


proxy: 177.54.97.249:3128
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
proxy: http://177.54.97.249:3128成功获得数据
